In [1]:
from rdkit import Chem
print(Chem.MolFromSmiles('c1ccccc1'))

In [12]:
conda install scikit-learn


Channels:
Note: you may need to restart the kernel to use updated packages.

 - defaults
 - conda-forge
Platform: win-64
Solving environment: ...working... done

# All requested packages already installed.



In [4]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Data process
import pandas as pd
import numpy as np
from sklearn.utils import resample
import os

import pandas as pd
import numpy as np
from sklearn.utils import resample
from rdkit import Chem
from rdkit.Chem.MolStandardize import rdMolStandardize

def process_chembl_data(filepath):
    # Read data
    x = pd.read_csv(filepath, sep=";")

    # Display basic data info
    print(x.head())
    print(x.shape)
    print(x.columns)

    # Select specific columns
    x1 = x[['Molecule ChEMBL ID', 'Smiles', 'Standard Type', 'Standard Relation', 'Standard Value', 'Standard Units']]
    print(x1.head())
    print(x1.shape)
    print(x1['Standard Units'].value_counts())
    print(x1['Standard Relation'].value_counts())
    print(x1['Standard Type'].value_counts())

    # Sorting and cleaning data
    x1.sort_values('Standard Units', ascending=True, inplace=True)
    x1 = x1.dropna()

    # Filtering data
    df = x1[x1['Standard Units'].str.contains('nM')]
    print(df['Standard Type'].value_counts())
    print(df['Molecule ChEMBL ID'].value_counts())

    # More data manipulation
    df3 = df[df['Molecule ChEMBL ID'].str.contains('CHEMBL488')]
    print(df3.head(20))
    print(df3.tail())
    print(df3['Standard Value'].min())

    df3.sort_values('Standard Value', ascending=True, inplace=True)
    print(df3)
    print(df.head())

    # Remove duplicates and unwanted standard types
    df.sort_values('Standard Value', ascending=True, inplace=True)
    df = df.drop_duplicates(subset=['Molecule ChEMBL ID'], keep='first')
    print(df.shape)
    df = df[df['Standard Type'].str.contains('IC50')]
    df = df[~df['Standard Type'].str.contains('pIC50|Log IC50')]
    print(df['Standard Type'].value_counts())

    # Classifying compounds
    active = df.loc[df['Standard Value'] <= 100]
    inactive = df.loc[df['Standard Value'] >= 1000]

    # Handling data imbalance
    combined_df = pd.concat([active, inactive])
    df_majority = combined_df[combined_df['Standard Value'] >= 1000]
    df_minority = combined_df[combined_df['Standard Value'] <= 100]

    df_minority_upsampled = resample(df_minority, replace=True, n_samples=len(df_majority), random_state=123)
    df_upsampled = pd.concat([df_majority, df_minority_upsampled])

    print(df_upsampled['Standard Value'].value_counts())

    # Further filtering
    df = df_upsampled[~df_upsampled['Standard Units'].str.contains('ug.mL-1')]
    active = df.loc[df['Standard Value'] <= 100]
    inactive = df.loc[df['Standard Value'] >= 1000]

    active = active.assign(label=1)
    inactive = inactive.assign(label=0)

    # Combine and save results
    combined = pd.concat([active, inactive])
    combined.to_csv('aromatase_filtered.csv', index=False)

    # Standardize SMILES
    def standardize_smiles(smiles):
        """Standardizes a SMILES string using RDKit."""
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            Chem.rdmolops.RemoveStereochemistry(mol)
            mol = rdMolStandardize.ChargeParent(mol)
            return Chem.MolToSmiles(mol, isomericSmiles=False)
        return None

    combined['Standardized_Smiles'] = combined['Smiles'].apply(standardize_smiles)
    combined.to_csv('aromatase_standardized.csv', index=False)
    return combined

# Example usage:
df_processed = process_chembl_data('data.csv')


from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, cohen_kappa_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from rdkit import RDLogger
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict # Import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, cohen_kappa_score, confusion_matrix
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import (RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier,
                              StackingClassifier, BaggingClassifier)
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, cohen_kappa_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from rdkit import RDLogger
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict # Import cross_val_predict
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score, cohen_kappa_score, confusion_matrix
import pandas as pd
import numpy as np
from sklearn.utils import resample
from rdkit import Chem
# Import AllChem explicitly
from rdkit.Chem import AllChem
from rdkit.Chem.MolStandardize import rdMolStandardize
# prompt: create a new data  from aromatase_standardized.csv, select : Filter
# Molecule ChEMBL ID, Standarized_Smiles, label

import pandas as pd
new_df = pd.read_csv('aromatase_standardized.csv', usecols=['Molecule ChEMBL ID', 'Standardized_Smiles', 'label'])
new_df[['Standardized_Smiles','label']].to_csv("aromatase.smi", index=None, header =None, sep='\t')
t1 = Chem.SmilesMolSupplier('aromatase.smi', delimiter='\t', titleLine=False)
fp = [AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048) for mol in t1 if mol]
train = np.asarray(fp, dtype= np.int32)
ids = [mol.GetProp('_Name') for mol in t1 if mol]
labels = np.asarray(ids, dtype = int).reshape(-1,1) # Use the built-in int instead of np.int
dataset = np.hstack([train, labels])
np.save('dataset_feature', dataset)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(train, labels, test_size=0.25,shuffle= True,  random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(random_state=42, probability=True), # probability=True for ROC AUC
    "KNN": KNeighborsClassifier(),
    "MLP": MLPClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss'),
    "Naive Bayes": GaussianNB(),
    "Logistic Regression": LogisticRegression(random_state=42, max_iter=1000),
    "Bagging": BaggingClassifier(random_state=42)
}

results_baseline_models = {}
for name, model in models.items():
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    y_prob = model.predict_proba(x_test)[:, 1]  # Probability for positive class

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob)
    mcc = matthews_corrcoef(y_test, y_pred)
    balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)

    # Calculate Sensibility and Specificity from the confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sensibility = tp / (tp + fn) if (tp + fn) !=0 else 0
    specificity = tn / (tn + fp) if (tn + fp) !=0 else 0

    results_baseline_models[name] = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'ROC AUC': roc_auc,
        'MCC': mcc,
        'Balanced Accuracy': balanced_accuracy,
        'Kappa': kappa,
        'Sensibility': sensibility,
        'Specificity': specificity
    }
    print(f"\n{name}:")
    for metric, value in results_baseline_models[name].items():
        print(f"{metric}: {value}")
    print(classification_report(y_test, y_pred))
# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame.from_dict(results_baseline_models, orient='index')

# Convert results to DataFrame and save
results_df = pd.DataFrame.from_dict(results_baseline_models, orient='index')
results_df.to_csv('baseline_models_results.csv', index=True)
# Function to calculate metrics
def calculate_metrics(y_true, y_pred, y_prob=None):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) != 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, zero_division=0),
        'Recall': recall_score(y_true, y_pred, zero_division=0),
        'F1-Score': f1_score(y_true, y_pred, zero_division=0),
        'ROC AUC': roc_auc_score(y_true, y_prob) if y_prob is not None else None,
        'MCC': matthews_corrcoef(y_true, y_pred),
        'Balanced Accuracy': balanced_accuracy_score(y_true, y_pred),
        'Kappa': cohen_kappa_score(y_true, y_pred),
        'Sensitivity': sensitivity,
        'Specificity': specificity
    }

# Define the base models for stacking
base_models = [(name, model) for name, model in models.items()]

# Define final estimator for stacking
final_estimator = LogisticRegression(random_state=42)

# Create the stacking classifier
stacking_classifier = StackingClassifier(
    estimators=base_models,
    final_estimator=final_estimator,
    cv=5,  # 5-fold stratified cross-validation
    stack_method='auto',
    n_jobs=-1
)

# Train and evaluate the stacked model
stacking_classifier.fit(x_train, y_train.ravel())
y_train_pred = stacking_classifier.predict(x_train)
y_train_proba = stacking_classifier.predict_proba(x_train)[:, 1]
y_test_pred = stacking_classifier.predict(x_test)
y_test_proba = stacking_classifier.predict_proba(x_test)[:, 1]

stack_train_metrics = calculate_metrics(y_train, y_train_pred, y_train_proba)
stack_test_metrics = calculate_metrics(y_test, y_test_pred, y_test_proba)

print("\nStacked Model - Training vs. Testing Performance Comparison:")
for metric in stack_train_metrics.keys():
    print(f"{metric} - Train: {stack_train_metrics[metric]:.4f}, Test: {stack_test_metrics[metric]:.4f}")

# Save the stacked model and metrics
joblib.dump(stacking_classifier, 'stacking_classifier.pkl')
stacked_results_df = pd.DataFrame([stack_train_metrics, stack_test_metrics], index=['Train', 'Test'])
stacked_results_df.to_csv('stacked_model_performance.csv')

  Molecule ChEMBL ID Molecule Name  Molecule Max Phase  Molecular Weight  \
0      CHEMBL3349931           NaN                 NaN            343.47   
1      CHEMBL2111947           NaN                 NaN            315.50   
2       CHEMBL150448           NaN                 NaN            247.30   
3       CHEMBL289116   ROGLETIMIDE                 2.0            218.26   
4        CHEMBL39782           NaN                 NaN            232.28   

   #RO5 Violations  AlogP Compound Key  \
0              0.0   3.20           20   
1              0.0   3.90            3   
2              0.0   0.94           14   
3              0.0   1.17            1   
4              0.0   1.51           14   

                                              Smiles Standard Type  \
0  CC(=O)NC[C@]12CCC(=O)C=C1CC[C@@H]1[C@@H]2CC[C@...    Inhibition   
1  C[C@]12CC[C@H]3[C@@H](CC=C4C[C@@H](O)CC[C@@]43...          IC50   
2                   CCC1(c2ccc(N)cc2)CCC(=O)N(N)C1=O    Inhibition   
3         

C:\Users\yanni\AppData\Local\Temp\ipykernel_13284\657858648.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x1.sort_values('Standard Units', ascending=True, inplace=True)
C:\Users\yanni\AppData\Local\Temp\ipykernel_13284\657858648.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.sort_values('Standard Value', ascending=True, inplace=True)
C:\Users\yanni\AppData\Local\Temp\ipykernel_13284\657858648.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return


Random Forest:
Accuracy: 0.9510268562401264
Precision: 0.9319526627218935
Recall: 0.9752321981424149
F1-Score: 0.9531013615733738
ROC AUC: 0.986332767402377
MCC: 0.9029343400385023
Balanced Accuracy: 0.9505193248776591
Kappa: 0.9019168770149701
Sensibility: 0.9752321981424149
Specificity: 0.9258064516129032
              precision    recall  f1-score   support

           0       0.97      0.93      0.95       310
           1       0.93      0.98      0.95       323

    accuracy                           0.95       633
   macro avg       0.95      0.95      0.95       633
weighted avg       0.95      0.95      0.95       633



C:\Users\yanni\.conda\envs\chemenv\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



AdaBoost:
Accuracy: 0.8688783570300158
Precision: 0.8703703703703703
Recall: 0.8730650154798761
F1-Score: 0.8717156105100463
ROC AUC: 0.940107859782283
MCC: 0.7376327135672923
Balanced Accuracy: 0.8687905722560672
Kappa: 0.7376290281502345
Sensibility: 0.8730650154798761
Specificity: 0.864516129032258
              precision    recall  f1-score   support

           0       0.87      0.86      0.87       310
           1       0.87      0.87      0.87       323

    accuracy                           0.87       633
   macro avg       0.87      0.87      0.87       633
weighted avg       0.87      0.87      0.87       633



...


XGBoost:
Accuracy: 0.9478672985781991
Precision: 0.9215116279069767
Recall: 0.9814241486068112
F1-Score: 0.9505247376311845
ROC AUC: 0.9836612403874962
MCC: 0.8975331327274212
Balanced Accuracy: 0.9471636872066314
Kappa: 0.8955482106336912
Sensibility: 0.9814241486068112
Specificity: 0.9129032258064517
              precision    recall  f1-score   support

           0       0.98      0.91      0.94       310
           1       0.92      0.98      0.95       323

    accuracy                           0.95       633
   macro avg       0.95      0.95      0.95       633
weighted avg       0.95      0.95      0.95       633


Naive Bayes:
Accuracy: 0.8894154818325435
Precision: 0.8373333333333334
Recall: 0.9721362229102167
F1-Score: 0.8997134670487106
ROC AUC: 0.8876810146809149
MCC: 0.7887895536411988
Balanced Accuracy: 0.8876810146809149
Kappa: 0.7779882154882155
Sensibility: 0.9721362229102167
Specificity: 0.8032258064516129
              precision    recall  f1-score   support

    

C:\Users\yanni\.conda\envs\chemenv\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\yanni\.conda\envs\chemenv\lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Logistic Regression:
Accuracy: 0.9352290679304898
Precision: 0.9028571428571428
Recall: 0.978328173374613
F1-Score: 0.939078751857355
ROC AUC: 0.976620393488465
MCC: 0.8733736181391692
Balanced Accuracy: 0.9343253770098872
Kappa: 0.870175930048172
Sensibility: 0.978328173374613
Specificity: 0.8903225806451613
              precision    recall  f1-score   support

           0       0.98      0.89      0.93       310
           1       0.90      0.98      0.94       323

    accuracy                           0.94       633
   macro avg       0.94      0.93      0.93       633
weighted avg       0.94      0.94      0.94       633



C:\Users\yanni\.conda\envs\chemenv\lib\site-packages\sklearn\ensemble\_bagging.py:802: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Bagging:
Accuracy: 0.933649289099526
Precision: 0.9048991354466859
Recall: 0.9721362229102167
F1-Score: 0.9373134328358209
ROC AUC: 0.9679616498551883
MCC: 0.8695489818135601
Balanced Accuracy: 0.9328423050034955
Kappa: 0.867035429211595
Sensibility: 0.9721362229102167
Specificity: 0.8935483870967742
              precision    recall  f1-score   support

           0       0.97      0.89      0.93       310
           1       0.90      0.97      0.94       323

    accuracy                           0.93       633
   macro avg       0.94      0.93      0.93       633
weighted avg       0.94      0.93      0.93       633


Stacked Model - Training vs. Testing Performance Comparison:
Accuracy - Train: 0.9989, Test: 0.9542
Precision - Train: 0.9979, Test: 0.9375
Recall - Train: 1.0000, Test: 0.9752
F1-Score - Train: 0.9989, Test: 0.9560
ROC AUC - Train: 1.0000, Test: 0.9837
MCC - Train: 0.9979, Test: 0.9090
Balanced Accuracy - Train: 0.9990, Test: 0.9537
Kappa - Train: 0.9979, Test: 0.90

In [ ]:
import pandas as pd

try:
    new_data = pd.read_csv('Moringa.csv', encoding='utf-8')  # Try default 'utf-8' first
except UnicodeDecodeError:
    new_data = pd.read_csv('Moringa.csv', encoding='latin-1')  # Fallback to 'latin-1'

new_data

In [3]:
# prompt: use the stacked model to predict unknown smile data in dataframe

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import joblib

# Load the trained stacked model
stacking_classifier = joblib.load('stacking_classifier.pkl')

def predict_smiles(smiles):
    """Predicts the label for a given SMILES string using the loaded stacked model."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048), dtype=np.int32).reshape(1, -1)
        prediction = stacking_classifier.predict(fp)
        probability = stacking_classifier.predict_proba(fp)
        return prediction[0], probability
    else:
        return None, None

# Example usage with a DataFrame
# Use the new_data DataFrame for predictions
data = new_data  
predictions = []
probabilities = []

for smiles in data['Smiles']:
    prediction, probability = predict_smiles(smiles)
    predictions.append(prediction)
    probabilities.append(probability)

# Create a new DataFrame for predictions OR overwrite new_data
# new_data['prediction'] = predictions  # To overwrite the original new_data
# new_data['probability'] = probabilities
prediction_df = pd.DataFrame({'Smiles': data['Smiles'], 'prediction': predictions, 'probability': probabilities}) # To create a separate DataFrame

# Display the prediction DataFrame
prediction_df
# prompt: in prediction_df  select smile predicted 1

# Assuming prediction_df is already created as in your provided code

# Select rows where 'prediction' is 1
smile_predicted_1 = prediction_df[prediction_df['prediction'] == 1]

# Display the selected rows
smile_predicted_1

[10:03:50] DEPRECATION WARNING: please use MorganGenerator
[10:03:50] DEPRECATION WARNING: please use MorganGenerator
[10:03:50] DEPRECATION WARNING: please use MorganGenerator
[10:03:50] DEPRECATION WARNING: please use MorganGenerator
[10:03:50] DEPRECATION WARNING: please use MorganGenerator
[10:03:51] DEPRECATION WARNING: please use MorganGenerator
[10:03:51] DEPRECATION WARNING: please use MorganGenerator
[10:03:51] DEPRECATION WARNING: please use MorganGenerator
[10:03:51] DEPRECATION WARNING: please use MorganGenerator
[10:03:51] DEPRECATION WARNING: please use MorganGenerator
[10:03:51] DEPRECATION WARNING: please use MorganGenerator
[10:03:52] DEPRECATION WARNING: please use MorganGenerator
[10:03:52] DEPRECATION WARNING: please use MorganGenerator
[10:03:52] DEPRECATION WARNING: please use MorganGenerator
[10:03:52] DEPRECATION WARNING: please use MorganGenerator
[10:03:52] DEPRECATION WARNING: please use MorganGenerator
[10:03:52] DEPRECATION WARNING: please use MorganGenerat

,Smiles,prediction,probability
27,C1=CC(=CC=C1C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O,1,"[[0.04180587351220144, 0.9581941264877986]]"
46,CC1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=C(OC4=CC(=CC(...,1,"[[0.4179652359955468, 0.5820347640044532]]"
47,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)OC4...,1,"[[0.3180500944942003, 0.6819499055057997]]"
48,CC1C(C(C(C(O1)OC2=C(OC3=CC(=CC(=C3C2=O)O)O)C4=...,1,"[[0.2982172867787164, 0.7017827132212836]]"
51,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O,1,"[[0.07275417741663048, 0.9272458225833695]]"
53,COC1=CC=C(C=C1)C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O,1,"[[0.20464242319497183, 0.7953575768050282]]"
54,C1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=C(OC4=CC(=CC(=...,1,"[[0.35670127165326904, 0.643298728346731]]"
56,COC1=C(C=CC(=C1)C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)...,1,"[[0.35640514847296356, 0.6435948515270364]]"
58,\nC1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=C(OC4=CC(=CC...,1,"[[0.35670127165326904, 0.643298728346731]]"
59,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)OC4...,1,"[[0.3180500944942003, 0.6819499055057997]]"


In [ ]:
# prompt: in prediction_df  select smile predicted 1

# Assuming prediction_df is already created as in your provided code

# Select rows where 'prediction' is 1
smile_predicted_1 = prediction_df[prediction_df['prediction'] == 1]

# Display the selected rows
smile_predicted_1
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw
# Assuming prediction_df is already created as in your provided code

# Select rows where 'prediction' is 1
smile_predicted_1 = prediction_df[prediction_df['prediction'] == 1]

# Display the selected rows
smile_predicted_1

# Assuming you want to display the first 10 SMILES with prediction 1 as images
# Check if there are at least 10 SMILES
num_smiles_to_display = min(10, len(smile_predicted_1))

# Create a list of molecules for the images
mols_to_draw = [Chem.MolFromSmiles(smi) for smi in smile_predicted_1['Smiles'].iloc[0:num_smiles_to_display]]

# Draw the molecules as a grid image
Draw.MolsToGridImage(mols_to_draw, molsPerRow=5) # Adjust molsPerRow for different grid layouts

In [8]:
import streamlit as st
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import joblib

# Load the model
model = joblib.load('stacking_classifier.pkl')

def predict_smiles(smiles):
    """Predicts the label for a given SMILES string using the loaded model."""
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        fp = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048), dtype=np.int32).reshape(1, -1)
        prediction = model.predict(fp)
        probability = model.predict_proba(fp)[0][1]  # Probability of being active
        return prediction[0], probability
    else:
        return None, None

def main():
    st.title('Phytochemical Screening App')
    st.write("This application uses a machine learning model to predict the activity of phytochemicals.")
    
    smiles_input = st.text_input("Enter SMILES string:", "")
    if st.button("Predict"):
        if smiles_input:
            prediction, probability = predict_smiles(smiles_input)
            if prediction is not None:
                st.success(f"Prediction: {'Active' if prediction == 1 else 'Inactive'}")
                st.write(f"Probability of being active: {probability:.4f}")
            else:
                st.error("Invalid SMILES string provided.")
        else:
            st.error("Please enter a SMILES string.")

if __name__ == "__main__":
    main()


NameError: name 'run_streamlit' is not defined

In [17]:
import pickle
filename = 'stacking_classifier.sav'
pickle.dump(stacking_classifier, open(filename, 'wb'))

In [18]:
# loading the save model
loaded_model = pickle.load(open('stacking_classifier.sav', 'rb'))

In [9]:
# prompt: use the stacked model to predict unknown smile data in dataframe

import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import joblib

# Load the trained stacked model
stacking_classifier = joblib.load('stacking_classifier.pkl')

def predict_smiles(smiles):
    """Predicts the label for a given SMILES string using the loaded stacked model."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        fp = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048), dtype=np.int32).reshape(1, -1)
        prediction = loaded_model.predict(fp)
        probability = loaded_model.predict_proba(fp)
        return prediction[0], probability
    else:
        return None, None

# Example usage with a DataFrame
# Use the new_data DataFrame for predictions
data = new_data  
predictions = []
probabilities = []

for smiles in data['Smiles']:
    prediction, probability = predict_smiles(smiles)
    predictions.append(prediction)
    probabilities.append(probability)

# Create a new DataFrame for predictions OR overwrite new_data
# new_data['prediction'] = predictions  # To overwrite the original new_data
# new_data['probability'] = probabilities
prediction_df = pd.DataFrame({'Smiles': data['Smiles'], 'prediction': predictions, 'probability': probabilities}) # To create a separate DataFrame

# Display the prediction DataFrame
prediction_df
# prompt: in prediction_df  select smile predicted 1

# Assuming prediction_df is already created as in your provided code

# Select rows where 'prediction' is 1
smile_predicted_1 = prediction_df[prediction_df['prediction'] == 1]

# Display the selected rows
smile_predicted_1

[10:20:56] DEPRECATION WARNING: please use MorganGenerator
[10:20:56] DEPRECATION WARNING: please use MorganGenerator
[10:20:56] DEPRECATION WARNING: please use MorganGenerator
[10:20:56] DEPRECATION WARNING: please use MorganGenerator
[10:20:56] DEPRECATION WARNING: please use MorganGenerator
[10:20:56] DEPRECATION WARNING: please use MorganGenerator
[10:20:56] DEPRECATION WARNING: please use MorganGenerator
[10:20:57] DEPRECATION WARNING: please use MorganGenerator
[10:20:57] DEPRECATION WARNING: please use MorganGenerator
[10:20:57] DEPRECATION WARNING: please use MorganGenerator
[10:20:57] DEPRECATION WARNING: please use MorganGenerator
[10:20:57] DEPRECATION WARNING: please use MorganGenerator
[10:20:57] DEPRECATION WARNING: please use MorganGenerator
[10:20:58] DEPRECATION WARNING: please use MorganGenerator
[10:20:58] DEPRECATION WARNING: please use MorganGenerator
[10:20:58] DEPRECATION WARNING: please use MorganGenerator
[10:20:58] DEPRECATION WARNING: please use MorganGenerat

,Smiles,prediction,probability
27,C1=CC(=CC=C1C2=CC(=O)C3=C(C=C(C=C3O2)O)O)O,1,"[[0.04180587351220144, 0.9581941264877986]]"
46,CC1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=C(OC4=CC(=CC(...,1,"[[0.4179652359955468, 0.5820347640044532]]"
47,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)OC4...,1,"[[0.3180500944942003, 0.6819499055057997]]"
48,CC1C(C(C(C(O1)OC2=C(OC3=CC(=CC(=C3C2=O)O)O)C4=...,1,"[[0.2982172867787164, 0.7017827132212836]]"
51,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O)O)O,1,"[[0.07275417741663048, 0.9272458225833695]]"
53,COC1=CC=C(C=C1)C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)O,1,"[[0.20464242319497183, 0.7953575768050282]]"
54,C1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=C(OC4=CC(=CC(=...,1,"[[0.35670127165326904, 0.643298728346731]]"
56,COC1=C(C=CC(=C1)C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)...,1,"[[0.35640514847296356, 0.6435948515270364]]"
58,\nC1C(C(C(C(O1)OCC2C(C(C(C(O2)OC3=C(OC4=CC(=CC...,1,"[[0.35670127165326904, 0.643298728346731]]"
59,C1=CC(=C(C=C1C2=C(C(=O)C3=C(C=C(C=C3O2)O)O)OC4...,1,"[[0.3180500944942003, 0.6819499055057997]]"


In [10]:
pip install flask


Note: you may need to restart the kernel to use updated packages.
